In [2]:
import psycopg2
import pandas as pd
#conn = psycopg2.connect(host='localhost', port=5432, dbname='city_db_test', user='postgres', password='postgres', application_name='select_maintenance')
conn = psycopg2.connect(host='10.32.1.107', port=5432, dbname='city_db_final', user='postgres', password='postgres', application_name='select_maintenance')
filename = 'maintenance_export_2023_12_05.xlsx'

In [2]:
from pandas import ExcelWriter


with conn, conn.cursor() as cur, ExcelWriter(filename, mode='w+') as writer:
    cur.execute('SELECT id, name, code FROM social_groups ORDER BY code')
    df = pd.DataFrame(cur.fetchall(), columns=[d.name for d in cur.description]).set_index('id')
    df.to_excel(writer, 'social_groups')
    
    cur.execute('SELECT id, name FROM living_situations ORDER BY id')
    df = pd.DataFrame(cur.fetchall(), columns=[d.name for d in cur.description]).set_index('id')
    df.to_excel(writer, 'living_situations')

    cur.execute('SELECT id, name FROM city_service_types ORDER BY id')
    df = pd.DataFrame(cur.fetchall(), columns=[d.name for d in cur.description]).set_index('id')
    df.to_excel(writer, 'city_service_types')

    cur.execute('SELECT id, name FROM maintenance.value_groups ORDER BY id')
    df = pd.DataFrame(cur.fetchall(), columns=[d.name for d in cur.description]).set_index('id')
    df.to_excel(writer, 'value_groups')

    cur.execute('SELECT id, name, (SELECT name from maintenance.value_groups WHERE id = group_id) group_name FROM maintenance.value_types ORDER BY id')
    df = pd.DataFrame(cur.fetchall(), columns=[d.name for d in cur.description]).set_index('id')
    df.to_excel(writer, 'value_types')

    cur.execute('SELECT sg.name social_group, st.name city_service_type FROM social_groups sg'
            ' JOIN maintenance.social_groups_city_service_types sgc ON sgc.social_group_id = sg.id'
            ' JOIN city_service_types st ON sgc.city_service_type_id = st.id'
            ' ORDER BY sg.code, st.id')
    df = pd.DataFrame(cur.fetchall(), columns=[d.name for d in cur.description])
    df.to_excel(writer, 'social_groups_city_service_types')
    
    cur.execute('SELECT sg.name social_group, vt.name value_type, ls.name living_situation FROM maintenance.social_groups_value_types_living_situations sgvtls'
            ' JOIN social_groups sg ON sgvtls.social_group_id = sg.id'
            ' JOIN maintenance.value_types vt ON sgvtls.value_type_id = vt.id'
            ' JOIN living_situations ls ON sgvtls.living_situation_id = ls.id'
            ' ORDER BY sg.id')
    df = pd.DataFrame(cur.fetchall(), columns=[d.name for d in cur.description])
    df.to_excel(writer, 'social_groups_value_types_living_situations')
    
    cur.execute('SELECT  cst.name city_service_name, ls.name living_situation FROM maintenance.living_situations_city_service_types lscst'
            ' JOIN city_service_types cst ON lscst.city_service_type_id = cst.id'
            ' JOIN living_situations ls ON lscst.living_situation_id = ls.id'
            ' ORDER BY ls.id')
    df = pd.DataFrame(cur.fetchall(), columns=[d.name for d in cur.description])
    df.to_excel(writer, 'living_situation_city_service_type')


C:\Users\OEM\AppData\Local\Programs\Python\Python311\Lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
